# Oráculo de Calificación

En este notebook implementamos una rutina de "Oráculo" que define si un estudiante aprueba o reprueba basándose en reglas heurísticas específicas, no solo en el promedio.

Estas reglas consideran el desempeño en el **item definitorio** (ej. Examen Final) y su relación con los **items avanzados** y otros componentes.

In [57]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import notas as nu
import config as cf

np.random.seed(42)
pd.set_option('display.float_format', '{:.2f}'.format)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Configuración y Generación de Datos
Generamos un conjunto de datos grande (N=500) para tener suficientes casos de prueba.

In [58]:
config = nu.autoconfigura_items(cf.config_evaluacion)
df_notas = nu.genera_datos(config, N=500)
display(df_notas.head())

Autoconfigurando items correlacionados...
 > Quices_Mod1: Ajustado vs Tarea_Mod1 (x0.5)
 > Quices_Mod2: Ajustado vs Quices_Mod1 (x1.2)
 > Tarea_Mod2: Ajustado vs Tarea_Mod1 (x1.2)
 > Quices_Mod3: Ajustado vs Quices_Mod1 (x1.0)
 > Tarea_Mod3: Ajustado vs Tarea_Mod1 (x1.0)


,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final
0,3.68,2.24,4.09,1.91,2.06,2.61,1.74
1,3.15,1.80,3.59,1.13,4.11,3.55,3.28
2,4.44,2.42,4.28,1.97,1.29,4.10,1.59
3,4.28,1.87,3.52,2.94,4.41,3.25,2.11
4,2.59,2.99,2.47,0.72,3.89,3.29,0.86


In [59]:
df_notas.describe()

,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final
count,500.00,500.00,500.00,500.00,500.00,500.00,500.00
mean,3.70,3.00,3.68,2.37,3.67,3.06,2.72
std,0.78,0.98,0.81,1.48,0.80,1.01,1.06
min,0.53,0.30,0.00,0.00,0.00,0.07,0.00
25%,3.37,2.35,3.43,1.10,3.36,2.39,2.02
50%,3.92,2.97,3.90,1.87,3.89,3.09,2.73
75%,4.28,3.66,4.28,3.88,4.26,3.74,3.38
max,4.50,5.00,4.50,5.00,4.50,5.00,5.00


## 2. Definición del Oráculo

Implementamos la función `calcula_decision_oraculo` con las siguientes reglas:

1. **Definitorio < 2.0**: Reprueba (0).
2. **2.0 <= Definitorio < 2.5**: Aprueba (1) SI todos los *items avanzados* > 3.0.
3. **2.5 <= Definitorio < 3.0**: Aprueba (1) SI al menos 2 *items avanzados* > 3.0.
4. **3.0 <= Definitorio < 3.5**: Reprueba (0) SI al menos 2 *items avanzados* < 3.0.
5. **3.5 <= Definitorio < 4.0**: Reprueba (0) SI todos los *items normales* (filtro) están perdidos (< 3.0).
6. **4.0 <= Definitorio < 5.0**: Reprueba (0) SI todos los *items avanzados* están perdidos (< 3.0).
7. **En otro caso**: Aprueba (1).

In [60]:
def calcula_decision_oraculo(row, config):
    # Identificar items
    def_item_name = config['item_definitorio']['nombre']
    items_normales = config['items_normales']
    
    # Separar por tipos
    # Asumimos que 'items de filtro' se refiere a todos los items normales
    names_avanzados = [i['nombre'] for i in items_normales if i.get('tipo') == 'avanzado']
    names_todos_normales = [i['nombre'] for i in items_normales]
    
    # Valores
    val_def = row[def_item_name]
    vals_adv = row[names_avanzados]
    vals_all_norm = row[names_todos_normales]
    
    # Umbral de aprobación estandár para chequeos internos
    PASS_THRESH = 2.95
    
    # 1. Def < 2.0 -> Perder
    if val_def < 2.0:
        return 0
    
    # 2. 2.0 <= Def < 2.5 -> Ganar SI todos avanzados > PASS
    elif 2.0 <= val_def < 2.5:
        if (vals_adv > PASS_THRESH).all():
            return 1
        return 0 # Si no cumple, pierde (asumo default behavior negativo en rangos bajos)
        
    # 3. 2.5 <= Def < 3.0 -> Ganar SI al menos 2 avanzados > PASS
    elif 2.5 <= val_def < 3.0:
        if (vals_adv > PASS_THRESH).sum() >= 2:
            return 1
        return 0

    # 4. 3.0 <= Def < 3.5 -> Perder SI al menos 2 avanzados < PASS
    elif 3.0 <= val_def < 3.5:
        if (vals_adv < PASS_THRESH).sum() >= 2:
            return 0
        return 1 # Default positivo en rango medio

    # 5. 3.5 <= Def < 4.0 -> Perder SI todos items normales perdidos
    elif 3.5 <= val_def < 4.0:
        if (vals_all_norm < PASS_THRESH).all():
            return 0
        return 1

    # 6. 4.0 <= Def < 5.0 -> Perder SI todos avanzados perdidos
    elif 4.0 <= val_def <= 5.0:
        if (vals_adv < PASS_THRESH).all():
            return 0
        return 1
        
    return 1

## 3. Aplicar Oráculo y Calcular Promedios

Calculamos las diferentes métricas para comparar y marcamos los **Falsos Positivos** (cuando el método aprueba pero el oráculo reprueba) y **Falsos Negativos** (cuando el método reprueba pero el oráculo aprueba).

In [61]:
# Calcular decisiones del oráculo
decisiones = df_notas.apply(lambda row: calcula_decision_oraculo(row, config), axis=1)

# Calcular notas mediante métodos existentes
df_res = df_notas.copy()
df_res['Decision_Oraculo'] = decisiones

# 1. Promedios y Notas
df_adv = nu.calcula_promedio_con_umbrales_avanzado(df_notas, config)
df_res['Promedio_Simple'] = df_adv['Promedio_Clasico']
df_res['Nota_Avanzada'] = df_adv['Nota_Final']
df_res['Nota_Umbral_Simple'] = nu.calcula_promedio_con_umbrales_simple(df_notas, config)

# 2. Calcular Falsos Positivos (FP) -> Oraculo=0, Nota>=3.0
df_res['FP_Promedio'] = ((df_res['Decision_Oraculo'] == 0) & (df_res['Promedio_Simple'] >= 3.0)).astype(int)
df_res['FP_Umbral_Simple'] = ((df_res['Decision_Oraculo'] == 0) & (df_res['Nota_Umbral_Simple'] >= 3.0)).astype(int)
df_res['FP_Avanzada'] = ((df_res['Decision_Oraculo'] == 0) & (df_res['Nota_Avanzada'] >= 3.0)).astype(int)

# 3. Calcular Falsos Negativos (FN) -> Oraculo=1, Nota<3.0
df_res['FN_Promedio'] = ((df_res['Decision_Oraculo'] == 1) & (df_res['Promedio_Simple'] < 3.0)).astype(int)
df_res['FN_Umbral_Simple'] = ((df_res['Decision_Oraculo'] == 1) & (df_res['Nota_Umbral_Simple'] < 3.0)).astype(int)
df_res['FN_Avanzada'] = ((df_res['Decision_Oraculo'] == 1) & (df_res['Nota_Avanzada'] < 3.0)).astype(int)

# 4. Reordenar Columnas
cols_calculadas = [
    'Decision_Oraculo', 'Promedio_Simple', 'Nota_Avanzada', 'Nota_Umbral_Simple', 
    'FP_Promedio', 'FP_Umbral_Simple', 'FP_Avanzada',
    'FN_Promedio', 'FN_Umbral_Simple', 'FN_Avanzada'
]
cols_items = [c for c in df_res.columns if c not in cols_calculadas]

cols_final = cols_items + ['Decision_Oraculo']
# Agregamos bloques por método
cols_final += ['Promedio_Simple', 'FP_Promedio', 'FN_Promedio']
cols_final += ['Nota_Umbral_Simple', 'FP_Umbral_Simple', 'FN_Umbral_Simple']
cols_final += ['Nota_Avanzada', 'FP_Avanzada', 'FN_Avanzada']

df_res = df_res[cols_final]

# Mostrar resumen
display(df_res.head(15))

,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final,Decision_Oraculo,Promedio_Simple,FP_Promedio,FN_Promedio,Nota_Umbral_Simple,FP_Umbral_Simple,FN_Umbral_Simple,Nota_Avanzada,FP_Avanzada,FN_Avanzada
0,3.68,2.24,4.09,1.91,2.06,2.61,1.74,0,2.36,0,0,1.89,0,0,2.36,0,0
1,3.15,1.80,3.59,1.13,4.11,3.55,3.28,1,2.79,0,1,2.23,0,1,2.79,0,1
2,4.44,2.42,4.28,1.97,1.29,4.10,1.59,0,2.59,0,0,2.07,0,0,2.59,0,0
3,4.28,1.87,3.52,2.94,4.41,3.25,2.11,0,2.86,0,0,2.29,0,0,2.86,0,0
4,2.59,2.99,2.47,0.72,3.89,3.29,0.86,0,2.02,0,0,1.62,0,0,2.02,0,0
5,4.17,3.18,4.34,0.00,4.12,4.24,3.97,1,3.24,0,0,0.00,0,1,0.00,0,1
6,4.48,2.90,2.32,3.81,4.36,2.92,3.62,1,3.44,0,0,3.05,0,0,3.31,0,0
7,4.29,3.27,3.77,1.32,3.47,4.04,3.13,1,3.13,0,0,1.60,0,1,2.25,0,1
8,4.44,2.47,4.36,3.55,4.30,3.10,2.95,0,3.35,1,0,2.82,0,0,3.21,1,0
9,3.30,4.55,3.47,1.46,4.49,4.13,2.64,1,3.20,0,0,1.79,0,1,2.44,0,1


### Informe de Desempeño (Rates)

Reportamos:
- **FPR (False Positive Rate)**: % de Negativos Reales que fueron aprobados incorrectamente.
- **FNR (False Negative Rate)**: % de Positivos Reales que fueron reprobados incorrectamente.

In [62]:
# Totales Reales
total_negatives = len(df_res[df_res['Decision_Oraculo'] == 0])
total_positives = len(df_res[df_res['Decision_Oraculo'] == 1])

print(f"Total Estudiantes: {len(df_res)}")
print(f"  > Positivos Reales (Oráculo=1): {total_positives}")
print(f"  > Negativos Reales (Oráculo=0): {total_negatives}")
print("=" * 60)

results = []
methods = [
    ('Promedio Simple', 'FP_Promedio', 'FN_Promedio'),
    ('Umbral Simple', 'FP_Umbral_Simple', 'FN_Umbral_Simple'),
    ('Umbral Avanzado', 'FP_Avanzada', 'FN_Avanzada')
]

for name, fp_col, fn_col in methods:
    n_fp = df_res[fp_col].sum()
    n_fn = df_res[fn_col].sum()
    fpr = n_fp / total_negatives if total_negatives > 0 else 0
    fnr = n_fn / total_positives if total_positives > 0 else 0
    results.append({
        'Método': name,
        'FPR': f"{fpr:.2%} ({n_fp})",
        'FNR': f"{fnr:.2%} ({n_fn})"
    })

df_results = pd.DataFrame(results)
display(df_results)

Total Estudiantes: 500
  > Positivos Reales (Oráculo=1): 230
  > Negativos Reales (Oráculo=0): 270


,Método,FPR,FNR
0,Promedio Simple,38.89% (105),16.52% (38)
1,Umbral Simple,1.85% (5),76.52% (176)
2,Umbral Avanzado,17.41% (47),64.35% (148)


### Visualización de Casos (Samples) por Método

A continuación mostramos ejemplos de Coincidencias, Falsos Positivos y Falsos Negativos para **cada uno de los tres métodos** comparados con el Oráculo.

In [63]:
num_show = 20

methods_config = [
    ("PROMEDIO SIMPLE", 'FP_Promedio', 'FN_Promedio'),
    ("UMBRAL SIMPLE", 'FP_Umbral_Simple', 'FN_Umbral_Simple'),
    ("UMBRAL AVANZADO", 'FP_Avanzada', 'FN_Avanzada')
]

for method_name, fp_col, fn_col in methods_config:
    print(f"\n{'#'*60}\n# EJEMPLOS PARA: {method_name}\n{'#'*60}")
    
    # 1. Matches
    df_matches = df_res[(df_res[fp_col] == 0) & (df_res[fn_col] == 0)]
    print(f"\n>>> {method_name}: COINCIDENCIAS ({len(df_matches)} / {len(df_res)})")
    if not df_matches.empty:
        display(df_matches.sample(min(num_show, len(df_matches))))
    else:
        print("No hay coincidencias.")

    # 2. FP
    df_fp = df_res[df_res[fp_col] == 1]
    print(f"\n>>> {method_name}: FALSOS POSITIVOS ({len(df_fp)} / {len(df_res)})")
    if not df_fp.empty:
        display(df_fp.sample(min(num_show, len(df_fp))))
    else:
        print("No hay Falsos Positivos.")

    # 3. FN
    df_fn = df_res[df_res[fn_col] == 1]
    print(f"\n>>> {method_name}: FALSOS NEGATIVOS ({len(df_fn)} / {len(df_res)})")
    if not df_fn.empty:
        display(df_fn.sample(min(num_show, len(df_fn))))
    else:
        print("No hay Falsos Negativos.")


############################################################
# EJEMPLOS PARA: PROMEDIO SIMPLE
############################################################

>>> PROMEDIO SIMPLE: COINCIDENCIAS (357 / 500)


,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final,Decision_Oraculo,Promedio_Simple,FP_Promedio,FN_Promedio,Nota_Umbral_Simple,FP_Umbral_Simple,FN_Umbral_Simple,Nota_Avanzada,FP_Avanzada,FN_Avanzada
307,3.81,3.48,4.03,0.99,3.48,2.70,3.31,1,3.00,0,0,2.35,0,1,3.00,0,0
498,3.53,4.41,3.87,4.12,3.53,2.55,5.00,1,4.06,0,0,3.51,0,0,3.86,0,0
41,3.84,4.12,3.99,1.27,4.37,2.48,3.54,1,3.18,0,0,1.33,0,1,2.07,0,1
66,3.85,2.00,3.81,1.06,3.32,3.18,4.35,1,3.07,0,0,0.89,0,1,1.64,0,1
222,4.21,2.75,1.86,2.09,4.39,4.36,2.00,0,2.82,0,0,2.25,0,0,2.82,0,0
162,4.15,4.03,3.43,1.48,4.44,3.70,2.50,1,3.08,0,0,1.75,0,1,2.39,0,1
140,4.38,2.01,4.06,0.28,4.01,4.59,2.32,0,2.68,0,0,2.15,0,0,2.68,0,0
318,4.42,3.49,4.33,1.60,4.44,3.93,2.06,1,3.03,0,0,1.59,0,1,2.35,0,1
461,4.46,1.70,4.48,3.94,4.18,2.79,3.51,1,3.42,0,0,1.92,0,1,2.66,0,1
376,3.86,2.56,4.02,0.76,3.78,3.75,0.75,0,2.23,0,0,1.78,0,0,2.23,0,0



>>> PROMEDIO SIMPLE: FALSOS POSITIVOS (105 / 500)


,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final,Decision_Oraculo,Promedio_Simple,FP_Promedio,FN_Promedio,Nota_Umbral_Simple,FP_Umbral_Simple,FN_Umbral_Simple,Nota_Avanzada,FP_Avanzada,FN_Avanzada
197,3.36,5.00,2.90,1.66,4.31,4.49,1.45,0,3.00,1,0,2.37,0,0,2.18,0,0
95,4.46,1.30,4.13,3.92,3.73,4.04,2.11,0,3.08,1,0,1.26,0,0,2.03,0,0
445,4.24,2.98,4.09,4.78,4.06,2.12,2.67,0,3.36,1,0,2.46,0,0,3.03,1,0
233,4.45,1.68,3.51,3.90,4.39,2.61,4.20,0,3.50,1,0,1.83,0,0,2.63,0,0
425,2.91,3.61,4.06,4.28,4.30,2.10,2.61,0,3.26,1,0,2.38,0,0,3.00,1,0
442,2.54,2.86,4.40,4.03,4.14,2.47,2.04,0,3.00,1,0,2.39,0,0,2.83,0,0
384,3.80,2.47,3.21,3.64,3.42,2.60,2.51,0,3.00,1,0,2.36,0,0,3.00,1,0
240,3.30,3.62,3.59,5.00,4.32,3.80,0.86,0,3.15,1,0,1.24,0,0,3.15,1,0
160,4.23,1.96,3.57,3.87,3.47,2.09,2.91,0,3.02,1,0,1.51,0,0,2.49,0,0
472,2.68,3.77,3.66,3.93,4.46,0.93,2.46,0,3.00,1,0,2.37,0,0,1.62,0,0



>>> PROMEDIO SIMPLE: FALSOS NEGATIVOS (38 / 500)


...

,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final,Decision_Oraculo,Promedio_Simple,FP_Promedio,FN_Promedio,Nota_Umbral_Simple,FP_Umbral_Simple,FN_Umbral_Simple,Nota_Avanzada,FP_Avanzada,FN_Avanzada
281,4.13,4.50,2.58,1.85,4.48,4.63,1.44,0,3.00,1,0,2.39,0,0,2.35,0,0
113,3.27,1.88,4.23,2.11,4.07,1.69,4.80,0,3.19,1,0,0.98,0,0,2.15,0,0
210,4.22,3.82,4.17,3.87,4.47,4.39,3.04,1,3.81,0,0,3.81,0,0,3.81,0,0
430,1.90,2.02,3.16,1.87,3.72,2.70,2.72,0,2.51,0,0,2.01,0,0,2.51,0,0
109,4.40,3.10,4.28,0.60,4.46,1.66,0.91,0,2.13,0,0,1.70,0,0,2.13,0,0
104,4.49,2.64,3.18,0.79,4.36,1.65,2.70,0,2.49,0,0,1.99,0,0,2.49,0,0
338,3.83,1.76,4.39,1.65,2.82,1.57,2.16,0,2.30,0,0,1.84,0,0,2.30,0,0
368,3.68,2.54,3.41,1.62,4.50,2.85,4.13,0,3.18,1,0,1.62,0,0,2.41,0,0
2,4.44,2.42,4.28,1.97,1.29,4.10,1.59,0,2.59,0,0,2.07,0,0,2.59,0,0
402,3.99,3.38,4.19,0.34,2.30,2.53,1.18,0,2.13,0,0,1.70,0,0,2.13,0,0



>>> UMBRAL AVANZADO: FALSOS POSITIVOS (47 / 500)


,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final,Decision_Oraculo,Promedio_Simple,FP_Promedio,FN_Promedio,Nota_Umbral_Simple,FP_Umbral_Simple,FN_Umbral_Simple,Nota_Avanzada,FP_Avanzada,FN_Avanzada
448,3.71,3.29,4.35,5.00,3.53,2.29,2.04,0,3.24,1,0,2.17,0,0,3.00,1,0
127,2.67,2.14,4.13,4.27,0.81,1.71,3.37,0,3.00,1,0,2.34,0,0,3.00,1,0
308,2.86,4.91,3.80,1.94,4.24,3.36,1.57,0,3.00,1,0,2.32,0,0,3.00,1,0
128,3.14,2.76,3.55,3.99,4.45,2.64,2.36,0,3.08,1,0,2.45,0,0,3.00,1,0
181,4.16,2.58,4.48,2.75,3.18,5.00,2.56,0,3.30,1,0,2.85,0,0,3.20,1,0
374,4.23,1.59,3.96,3.65,2.69,3.17,2.34,0,3.00,1,0,2.32,0,0,3.00,1,0
8,4.44,2.47,4.36,3.55,4.30,3.10,2.95,0,3.35,1,0,2.82,0,0,3.21,1,0
174,3.70,3.93,4.45,3.82,4.20,3.64,1.84,0,3.33,1,0,2.59,0,0,3.33,1,0
394,3.23,3.11,4.37,4.44,2.79,0.95,2.47,0,3.00,1,0,2.36,0,0,3.00,1,0
390,4.24,2.30,4.41,4.52,4.48,3.17,2.61,0,3.42,1,0,2.70,0,0,3.18,1,0



>>> UMBRAL AVANZADO: FALSOS NEGATIVOS (148 / 500)


,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final,Decision_Oraculo,Promedio_Simple,FP_Promedio,FN_Promedio,Nota_Umbral_Simple,FP_Umbral_Simple,FN_Umbral_Simple,Nota_Avanzada,FP_Avanzada,FN_Avanzada
131,3.18,1.62,3.96,3.53,3.00,3.36,3.11,1,3.07,0,0,1.77,0,1,2.46,0,1
32,4.43,4.58,0.98,0.96,3.80,4.53,3.93,1,3.35,0,0,0.51,0,1,1.33,0,1
312,3.94,2.93,3.19,4.28,0.79,3.67,4.46,1,3.65,0,0,1.45,0,1,1.92,0,1
238,2.59,2.14,3.21,1.71,4.38,4.11,4.43,1,3.30,0,0,1.58,0,1,2.47,0,1
289,4.47,4.75,4.48,1.08,4.25,2.16,4.21,1,3.47,0,0,1.08,0,1,1.83,0,1
245,3.93,3.88,4.47,1.50,4.28,3.40,4.61,1,3.67,0,0,2.09,0,1,2.74,0,1
433,2.73,3.63,3.63,1.19,3.14,3.65,2.93,1,2.89,0,1,2.31,0,1,2.89,0,1
192,4.41,3.83,1.86,1.05,3.27,5.00,2.65,1,3.00,0,0,2.40,0,1,1.66,0,1
78,3.96,3.67,2.70,0.59,3.71,4.56,4.07,1,3.30,0,0,0.78,0,1,1.23,0,1
320,4.17,3.75,1.05,1.56,4.07,2.91,3.20,1,2.89,0,1,2.31,0,1,2.89,0,1
